In [ ]:
# Minimal Reproduction Notebook
Run baseline probes, then convert a record to a findings JSON and validate against the schema.


In [ ]:
%pip install -r ../requirements.txt
%pip install -U vllm


In [ ]:
import os, subprocess, sys, time
os.environ['MODEL_NAME'] = os.environ.get('MODEL_NAME', 'gpt-oss-20b')
# Launch local vLLM OpenAI server serving gpt-oss-20b (requires GPU)
cmd = [sys.executable, '-m', 'vllm.entrypoints.openai.api_server', '--model', os.environ['MODEL_NAME'], '--host', '127.0.0.1', '--port', '8000']
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
# Wait briefly for server startup
time.sleep(8)
os.environ['OPENAI_BASE_URL'] = 'http://127.0.0.1:8000/v1'
os.environ['OPENAI_API_KEY'] = 'local-sk'  # placeholder for local server
!python ../scripts/run_probes.py --prompts ../data/prompts/baseline.yaml --out ../outputs/run-baseline.jsonl --temperature 0.7 --max-tokens 256
proc.terminate()


In [ ]:
!python ../scripts/make_finding.py --title 'Sample Issue' --topic 'reward hacking' --run ../outputs/run-baseline.jsonl --index 0 --out ../findings/findings.1.json
!python ../scripts/validate_finding.py ../findings/findings.1.json ../findings.schema
